In [1]:
import os 
import csv
from library_dicom.dicom_processor.tools.folders import * 

In [ ]:
#Studies already done 

In [2]:
ahl_csv_path = '/media/deeplearning/LACIE SHARE/AHL_NIFTI/AHL2011_dMax_v4.csv'
with open(ahl_csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_file = []
    for row in reader :
        csv_file.append(row)
        
del csv_file[0] #enlever première ligne

In [6]:
studies = csv_file
print(len(studies))
print(studies[0])

711
['13011101431012', '2013_01_24', '1.2.124.113532.172.16.25.2.20130121.814.1220583', '325.44']


In [8]:
#Studies to do 
ahl_validated_dicom_directory = '/media/oncopole/DD 2To/AHL/AHL2011_Validated_DICOMS'
csv_path = '/media/deeplearning/LACIE SHARE/AHL_NIFTI/pet0.csv'
with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ';') #liste pour chaque ligne 
    csv_file = []
    for row in reader :
        csv_file.append(row)
        
del csv_file[0] #enlever première ligne

In [9]:
len(csv_file)

796

In [11]:
patient_id  = []
for row in csv_file : 
    if row[1] not in patient_id : 
        patient_id.append(row[1])
print(len(patient_id))

739


In [12]:
d_max = []
for row in studies : 
    d_max.append(row[0])
print(len(d_max))

711


In [13]:
to_do = []
for id_ in patient_id : 
    if id_ not in d_max : 
        to_do.append(id_)

In [15]:
dataset = []
for id_ in to_do : 
    for row in csv_file : 
        if id_ in row : 
            dataset.append(row)
            break

In [21]:
d = []
for data in dataset : 
    print(data[1])
    d.append(data[1])

13011101331006
13011101601004
13011101481009
13011101481010
13011101481011
53011101371002
53011101401009
13011103311013
13011101251021
13011101251016
13011101251010
53011106341001
13011101021008
13011103311005
13011101031024
13011101251025
13011101431001
53011106341002
53011106341005
53011106341004
53011106341006
53011106341007
13011101391002
13011101391005
13011101481003
13011101251027
53011101821010
53011101821011


In [20]:
print(len(dataset))

28


In [22]:
d

['13011101331006',
 '13011101601004',
 '13011101481009',
 '13011101481010',
 '13011101481011',
 '53011101371002',
 '53011101401009',
 '13011103311013',
 '13011101251021',
 '13011101251016',
 '13011101251010',
 '53011106341001',
 '13011101021008',
 '13011103311005',
 '13011101031024',
 '13011101251025',
 '13011101431001',
 '53011106341002',
 '53011106341005',
 '53011106341004',
 '53011106341006',
 '53011106341007',
 '13011101391002',
 '13011101391005',
 '13011101481003',
 '13011101251027',
 '53011101821010',
 '53011101821011']

In [8]:
done = []
for row in csv_file : 
    patient_id = row[1]
    for study in studies : 
        if patient_id in study : 
            done.append(row)

In [9]:
len(done)
for r in done : 
    csv_file.remove(r)

In [10]:
len(csv_file)

74

In [11]:
write_json_file('/media/oncopole/DD 2To/AHL', 'ahl_to_do', csv_file)

In [12]:
series_paths = get_series_path('/media/oncopole/DD 2To/AHL/AHL2011_Validated_DICOMS')
export_folder = '/media/oncopole/DD 2To/AHL'

In [13]:
len(series_paths)

3976

In [14]:
series_paths[0]

'/media/oncopole/DD 2To/AHL/AHL2011_Validated_DICOMS/13011101031011/1.2.840.113704.1.111.2552.1344321001.3/1.2.840.113704.1.111.2332.1344322265.8/1.2.840.113704.1.111.2332.1344322265.8'

In [16]:
check = []
for row in csv_file : 
    patient_id  = row[1]
    study_uid = row[2]
    liste = []
    for serie_path in series_paths : 
        if patient_id in serie_path and study_uid in serie_path:
            liste.append(serie_path)
    if liste != [] :
        check.append(liste)

In [17]:
len(check)

46

In [18]:
for liste in check : 
    print(len(liste)) 

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


In [19]:
check[0]

['/media/oncopole/DD 2To/AHL/AHL2011_Validated_DICOMS/13011101431012/1.2.124.113532.172.16.25.2.20130121.814.1220583/1.2.840.113704.1.111.3652.1359035909.11/imagys_AHL2011_13011101431012_ce32ad94-1872-4340-9e14-a2b6ec451cf0',
 '/media/oncopole/DD 2To/AHL/AHL2011_Validated_DICOMS/13011101431012/1.2.124.113532.172.16.25.2.20130121.814.1220583/1.3.46.670589.28.2.12.30.26421.36112.2.3508.0.1359080111/imagys_AHL2011_13011101431012_c72b9b8c-80b3-4803-a240-62a9ec6b6302']

In [20]:
write_json_file('/media/oncopole/DD 2To/AHL', 'dataset_ahl', check)

In [21]:
from library_dicom.dicom_processor.model.Series import Series

In [26]:
dataset = []
for study in check : 
    print(check.index(study))
    new_liste = []
    for path in study : 
        new_liste.append(path)
        serie_objet = Series(path)
        instances = serie_objet.get_instances_ordered()
        serie_tags = instances[0].get_series_tags()
        modal = serie_tags['Modality']
        new_liste.append(modal)
    serie_objet = Series(study[0])
    instances = serie_objet.get_instances_ordered()
    study_tags = instances[0].get_studies_tags()
    study_uid = study_tags['StudyInstanceUID']
    patient_tags = instances[0].get_patients_tags()
    patient_id = patient_tags['PatientID']
    new_liste.append(study_uid)
    new_liste.append('pet0')
    new_liste.append(patient_id)
    dataset.append(new_liste)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
21
21
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45


In [27]:
write_json_file('/media/oncopole/DD 2To/AHL', 'ahl_dataset', dataset)

In [29]:
dataset[0]

['/media/oncopole/DD 2To/AHL/AHL2011_Validated_DICOMS/13011101431012/1.2.124.113532.172.16.25.2.20130121.814.1220583/1.2.840.113704.1.111.3652.1359035909.11/imagys_AHL2011_13011101431012_ce32ad94-1872-4340-9e14-a2b6ec451cf0',
 'CT',
 '/media/oncopole/DD 2To/AHL/AHL2011_Validated_DICOMS/13011101431012/1.2.124.113532.172.16.25.2.20130121.814.1220583/1.3.46.670589.28.2.12.30.26421.36112.2.3508.0.1359080111/imagys_AHL2011_13011101431012_c72b9b8c-80b3-4803-a240-62a9ec6b6302',
 'PT',
 '1.2.124.113532.172.16.25.2.20130121.814.1220583',
 'pet0',
 '13011101431012']

In [30]:
csv_directory = '/media/oncopole/DD 2To/AHL/AHL2011_CSV_DATA/AHL2011-CSV'
for data in dataset : 
    study_uid = data[-3]
    for row in csv_file : 
        if study_uid == row[2] : 
            data.append(os.path.join(csv_directory, row[4]))

In [34]:
new = []
for data in dataset : 
    if data not in new : 
        new.append(data)

In [36]:
for data in new : 
    if len(data) != 8 : 
        print(data )

['/media/oncopole/DD 2To/AHL/AHL2011_Validated_DICOMS/13011101021018/1.2.840.113619.2.290.3.2500492204.214.1361944919.828/1.2.840.113619.2.290.3.2500492204.214.1361944919.833.3/1.2.840.113619.2.290.3.2500492204.214.1361944919.833.3', 'CT', '/media/oncopole/DD 2To/AHL/AHL2011_Validated_DICOMS/13011101021018/1.2.840.113619.2.290.3.2500492204.214.1361944919.828/1.2.840.113619.2.290.3.2500492204.214.1361944920.206/1.2.840.113619.2.290.3.2500492204.214.1361944920.206', 'PT', '1.2.840.113619.2.290.3.2500492204.214.1361944919.828', 'pet0', '13011101021018', '/media/oncopole/DD 2To/AHL/AHL2011_CSV_DATA/AHL2011-CSV/cro_ma_feb 27_2013_SK.csv', '/media/oncopole/DD 2To/AHL/AHL2011_CSV_DATA/AHL2011-CSV/cro_ma_feb 27_2013_abr.csv', '/media/oncopole/DD 2To/AHL/AHL2011_CSV_DATA/AHL2011-CSV/cro_ma_feb 27_2013-EV.csv']
['/media/oncopole/DD 2To/AHL/AHL2011_Validated_DICOMS/13011101391005/1.2.250.1.59.453.859.92444169.1436.1348829892.1.1.1/1.2.250.1.59.453.859.92444169.1436.1348829892.1.2.5/1.2.250.1.59.4

In [39]:
write_json_file('/media/oncopole/DD 2To/AHL', 'ahl_dataset', dataset)

In [40]:
len(new)

44

In [42]:
csv_path= '/media/oncopole/DD 2To/AHL/NIFTI/AHL2011_NIFTI.csv'
with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_file = []
    for row in reader :
        csv_file.append(row)
        
del csv_file[0] #enlever première ligne

In [43]:
csv_file[0]

['13011101431012',
 '13011101431012',
 '1.2.124.113532.172.16.25.2.20130121.814.1220583',
 'pet0',
 '/media/oncopole/DD 2To/AHL/NIFTI/13011101431012/pet0/1.2.124.113532.172.16.25.2.20130121.814.1220583_nifti_CT.nii',
 '/media/oncopole/DD 2To/AHL/NIFTI/13011101431012/pet0/1.2.124.113532.172.16.25.2.20130121.814.1220583_nifti_PT.nii',
 '/media/oncopole/DD 2To/AHL/NIFTI/13011101431012/pet0/1.2.124.113532.172.16.25.2.20130121.814.1220583_nifti_mask.nii']

In [44]:
json_path = '/media/oncopole/DD 2To/AHL/unconstant/unconstant_spacing.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

In [45]:
dataset[0]

['/media/oncopole/DD 2To/AHL/AHL2011_Validated_DICOMS/13011101431012/1.2.124.113532.172.16.25.2.20130121.814.1220583/1.2.840.113704.1.111.3652.1359035909.11/imagys_AHL2011_13011101431012_ce32ad94-1872-4340-9e14-a2b6ec451cf0',
 'CT',
 '/media/oncopole/DD 2To/AHL/AHL2011_Validated_DICOMS/13011101431012/1.2.124.113532.172.16.25.2.20130121.814.1220583/1.3.46.670589.28.2.12.30.26421.36112.2.3508.0.1359080111/imagys_AHL2011_13011101431012_c72b9b8c-80b3-4803-a240-62a9ec6b6302',
 'PT',
 '1.2.124.113532.172.16.25.2.20130121.814.1220583',
 'pet0',
 '13011101431012',
 '/media/oncopole/DD 2To/AHL/AHL2011_CSV_DATA/AHL2011-CSV/can_sy_jan 24_2013.csv']

In [51]:
unconstant = []
good = []
for row in csv_file : 
    patient_id = row[1]
    for data in dataset : 
        if patient_id in data : 
            row.append('unconstant')


In [59]:
unconstant = []
good = []
for row in csv_file : 
    if 'unconstant' in row : 
        unconstant.append(row)
    else : good.append(row)

In [61]:
len(unconstant)

22

In [63]:
good[0]

['13011101051002',
 '13011101051002',
 '1.2.250.1.74.20110825143500.60000017823632',
 'pet0',
 '/media/oncopole/DD 2To/AHL/NIFTI/13011101051002/pet0/1.2.250.1.74.20110825143500.60000017823632_nifti_CT.nii',
 '/media/oncopole/DD 2To/AHL/NIFTI/13011101051002/pet0/1.2.250.1.74.20110825143500.60000017823632_nifti_PT.nii',
 '/media/oncopole/DD 2To/AHL/NIFTI/13011101051002/pet0/1.2.250.1.74.20110825143500.60000017823632_nifti_mask.nii']

In [65]:
directory = '/media/oncopole/DD 2To/AHL/NIFTI'
filename = 'AHL2011_NIFTI_unconstant.csv'

with open(os.path.join(directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT NAME", "PATIENT_ID", "STUDY UID", "PET", "NIFTI_CT", "NIFTI_PET", "NIFTI_MASK"])
    for serie in unconstant: 
        csv_writer.writerow([serie[0], serie[1], serie[2], serie[3], serie[4], serie[5], serie[6]])